In [17]:
import sys

import pandas as pd
import numpy as np
#import matplotlib as plt
#from matplotlib import pyplot
#import scipy as sp
#from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


import joblib

In [2]:
filename = 'ginf.csv'
games = pd.read_csv('./data/' + filename)


In [3]:
games["result"] = games['fthg'] - games['ftag']

col = 'result'
conditions = [games[col] > 0 , games[col] == 0, games[col] < 0]
choices = [1 , 2 , 3]
games['result'] = np.select(conditions,choices,default=np.nan)



games.head()

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n,result
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN,1.0
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN,2.0
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN,1.0
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN,3.0
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN,1.0


In [4]:
games_prediction = games.iloc[:,[5,7,8,-1]]
dummies = pd.get_dummies(games_prediction, columns=['season','ht','at'])
dummies.head()

,result,season_2012,season_2013,season_2014,season_2015,season_2016,season_2017,ht_AC Ajaccio,ht_AC Milan,ht_AJ Auxerre,...,at_Valenciennes,at_VfB Stuttgart,at_VfL Wolfsburg,at_Villarreal,at_Watford,at_Werder Bremen,at_West Brom,at_West Ham,at_Wigan,at_Wolves
0,1.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
X = dummies.iloc[:,1:]
y = dummies.iloc[:,:1]
print(X.shape)
print(y.shape)

(10112, 301)
(10112, 1)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [7]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestClassifier(n_estimators=100))

hyperparameters = { 'randomforestclassifier__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestclassifier__max_depth': [None, 5, 3, 1]}


clf = GridSearchCV(pipeline, hyperparameters, cv=10)

In [8]:
#clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(30,20,3), random_state=1 , max_iter = 1000)
#clf = KNeighborsClassifier(n_neighbors=15, algorithm='auto');
##clf = RandomForestClassifier(n_estimators=400)
clf.fit(X_train,y_train)

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklea

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklea

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklea

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklea

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklea

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                  

In [9]:
y_pred = clf.predict(X_test)

print(r2_score(y_test, y_pred))

print(mean_squared_error(y_test, y_pred))


-0.5900394154948323
1.1387607119314436


In [10]:
accuracy = clf.score(X_test, y_test)
#y_pred = clf.predict_proba(X_test)
predict = clf.predict(X_test)
y_total = y_train.count()
y_positive = y_train.sum()
print('The training set contains {} examples (games) of which {} are positives .'.format(y_total, y_positive))
print('The accuracy of classifying is {:.2f} %'.format(accuracy*100))

The training set contains result    7078
dtype: int64 examples (games) of which result    12914.0
dtype: float64 are positives .
The accuracy of classifying is 46.74 %


In [11]:
game = 100
labels = X_test.columns.values

for x in range(len(X_test.iloc[game])): 
    if X_test.iloc[game][x] == 1:
        print(labels[x])



season_2013
ht_Stade de Reims
at_Valenciennes


In [12]:
print(y_test.columns.values)
clf.predict([X_test.iloc[game]])

['result']


array([1.])

In [13]:
y_test.iloc[game]

result    3.0
Name: 3397, dtype: float64

In [18]:
joblib.dump(clf, './models/rfr_predict_winner.pkl')

['./models/rfr_predict_winner.pkl']